In [41]:
import tensorflow as tf
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))

%matplotlib inline

In [42]:
features = ['Malignancy', 'Diameter', 'Lobulation', 'Spiculation']

In [43]:
%run 'CNNBlocks.ipynb'
%run 'modelArchitectureRegression.ipynb'
%run 'batchGeneratorRegression.ipynb'

In [44]:
validInd = {}
for key in ['random', 'true']:
    validInd[key] = np.load('/home/katya/LungCancer/Katya/CNN_v3_nodule_classification/model_and_weights/valid_inds_'+key+'.npy')

In [45]:
modelPath = '/home/katya/LungCancer/Katya/CNN_v3_nodule_classification/model_and_weights/LUNA_model_v3_regression.h5'
modelPathClass = '/home/katya/LungCancer/Katya/CNN_v3_nodule_classification/model_and_weights/LUNA_model_v3_class.h5'

In [26]:
print ('Loading positive patches')
xPosTrain, ixPosTrain, xPosValid, ixPosValid = nodulePredictor('true', modelPathClass, validInd)

print ('Loading negative patches')
xNegTrain, ixNegTrain, xNegValid, ixNegValid = nodulePredictor('random', modelPathClass, validInd)

Loading positive patches


/opt/anaconda/lib/python3.5/site-packages/keras/engine/topology.py:1206: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


The number of samples for true category constitutes 3570
Predicting...
2856/2856 [==============================] - 152s     

In [46]:
#Compiling model with branching on the level of Convolution block #5
model = compileModelDeepBranching((1,64,64,64),dropRate=0.2,regRate=1e-3)

/opt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(8, (3, 3, 3), padding="same", kernel_regularizer=<keras.reg..., data_format="channels_first")`
/opt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:12: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(data_format="channels_first")`
/opt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: UserWarning: Update your `AveragePooling3D` call to the Keras 2 API: `AveragePooling3D(data_format="channels_first")`
/opt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/opt/anaconda/lib/python3.5/site-packages/keras/legacy/layers.py:456: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras

Compiling model...


In [ ]:
nbEpochs = 10

model, lossHist = trainRegressionModel(model, modelPath=modelPath, modelPathClass=modelPathClass,
                                       validInd=validInd, 
                                       batchSize=30, nbEpoch=nbEpochs, stepsPerEpoch=50)

model.save_weights('/home/katya/LungCancer/Katya/CNN_v3_nodule_classification/model_and_weights/LUNA_model_v3_weights_regression.h5')

/opt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:45: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., callbacks=[<keras.ca..., validation_steps=20, steps_per_epoch=50, epochs=1, initial_epoch=0, validation_data=<generator...)`


Epoch 1/1
50/50 [==============================] - 142s - loss: 1.4100 - Malignancy_loss: 0.1057 - Diameter_loss: 0.0847 - Lobulation_loss: 0.0940 - Spiculation_loss: 0.1996 - val_loss: 1.0722 - val_Malignancy_loss: 0.1429 - val_Diameter_loss: 0.0340 - val_Lobulation_loss: 0.0544 - val_Spiculation_loss: 0.0490

/opt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:45: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., callbacks=[<keras.ca..., validation_steps=20, steps_per_epoch=50, epochs=2, initial_epoch=1, validation_data=<generator...)`


49/50 [============================>.] - ETA: 2s - loss: 0.8791 - Malignancy_loss: 0.0980 - Diameter_loss: 0.0353 - Lobulation_loss: 0.0479 - Spiculation_loss: 0.0521  

In [ ]:
plt.figure(figsize=[10,7])
plt.grid(True, ls='--', lw=0.5, alpha=0.5, dash_capstyle = 'round', c='gray')
plt.xlabel('Epoch #', fontsize=15)
plt.ylabel('Loss', fontsize=15)

labels=list(lossHist.keys())

for i,key in enumerate(lossHist):
    plt.plot([x for x in range(nbEpochs)], lossHist[key], 'o-', label=labels[i])
    plt.legend()
    
plt.savefig('/home/katya/LungCancer/Katya/CNN_v3_nodule_classification/figures/' + str(version) + '.png')

In [ ]:
version = 2.2

In [ ]:
for feature in features:
    plt.figure(figsize=[10,8])
    plt.grid(True, ls='--', lw=0.5, alpha=0.5, dash_capstyle = 'round', c='gray')

    plt.plot([x for x in range(nbEpochs)], lossHist[feature+'_loss'], label=feature+'_loss')
    plt.plot([x for x in range(nbEpochs)], lossHist['val_'+feature+'_loss'], label='val_'+feature+'_loss')
    plt.title(feature)
    plt.legend()
    plt.savefig('/home/katya/LungCancer/Katya/CNN_v3_nodule_classification/figures/'+str(version)+'.'+feature+'.png')